Setting up parameters

In [47]:
FilesPath = 'C:\\Users\\styrn\\Documents\\GitHub\\Netter\\L2 Data'
TimeFrame = '5Min'	#Таймфрейм построения графика
Period = 24*12		#Анализируемый период
SmoothATR = 0.9		#Множитель "загрубления" ценовых уровней
AspectVA = 10.0		#Величина VAL | VAH в % от среднего объема торгов на уровне
		#Минимальный шаг сетки ордеров
		#Размер SL и TP как % от волатильности
		#Гибридный/трендовый/откатный режим.

Reading data from file with msgpackL2 package. And extracting L1 data as footprint.

In [ ]:
import numpy
from datetime import datetime
from os import listdir
import zlib
import msgpackL2

# decompressing and reading data from file
filelist = listdir(FilesPath)
print('Files in ', FilesPath, filelist)

# transmitting L2 data into footprint: datetime, price, traded_volume
footprint = numpy.empty(shape=(0,3))

for file in filelist:
    print('Processing file', file)
    data =  open(FilesPath+'\\'+file, 'rb').read()
    messages =  msgpackL2.unpackMsgs(zlib.decompress(data, 16+zlib.MAX_WBITS))

    for message in messages:   
        if message[4]:
            for trade in message[4]:
                footprint = numpy.vstack((footprint,  [datetime.fromtimestamp(message[0]), trade[0], trade[1]]))

print('Footprint shape', footprint.shape)

Files in  C:\Users\styrn\Documents\GitHub\Netter\L2 Data ['20170811', '20170812', '20170813', '20170814', '20170815']
Processing file 20170811
Processing file 20170812
Processing file 20170813
Processing file 20170814
Processing file 20170815


Data manipulation. Constructing ohlc dataframe, horizontal volume histogram, and net levels.

In [45]:
import numpy
import pandas

# transmitting data into ohlc candlestick struct: datetime, open_price, high, low, close, traded_volume
df = pandas.DataFrame(abs(footprint[0:,1:]), footprint[0:,0], ['prc','vol'])
ohlc = df.groupby(pandas.TimeGrouper(freq=TimeFrame)).agg({'prc':{'first','last','min','max'},'vol':'sum'})
# and horizontal volume distribution
hor_vol = df.groupby('prc').agg({'vol':'sum'})

# finding average volume by level, with net factor (adjusting parameter)
base_vol = hor_vol.mean().vol*AspectVA;

# slicing net_levels by VAL, VAH, VAB where hor_vol > base_vol
net_levels = hor_vol[hor_vol.gt(base_vol).all(axis=1)]
net_levels = net_levels.assign(VALH='n')

for level in net_levels.index:
    if base_vol > hor_vol.shift(1).loc[level,'vol']: # vol > vol_at_prev_level => VAL bond     
        net_levels.loc[level,'VALH'] = 'l'
    if base_vol > hor_vol.shift(-1).loc[level,'vol']: # vol > vol_at_next_level => VAH bond
        net_levels.loc[level,'VALH'] = 'h'
    if base_vol > hor_vol.shift(1).loc[level,'vol'] and base_vol > hor_vol.shift(-1).loc[level,'vol']: # VAB Both-side bond
        net_levels.loc[level,'VALH'] = 'b'    
net_levels = net_levels.drop(net_levels[net_levels.VALH =='n'].index)
        
print('Base volume',base_vol)
print(net_levels)

# calculating ATR 
atr = pandas.DataFrame(index=ohlc.index, columns=['max_min','max_cls','min_cls'])
atr['max_min'] = abs (ohlc.loc[:,'prc'].loc[:,'max'] - ohlc.loc[:,'prc'].loc[:,'min'])
atr['max_cls'] = abs (ohlc.loc[:,'prc'].loc[:,'max'] - ohlc.loc[:,'prc'].loc[:,'last'].shift())
atr['min_cls'] = abs (ohlc.loc[:,'prc'].loc[:,'min'] - ohlc.loc[:,'prc'].loc[:,'last'].shift())
ohlc['ATR'] = atr[['max_min', 'max_cls', 'min_cls']].max(axis=1)
#print(ohlc)

# smoothing it by EWM, SmoothATR - adjustable
ohlc['smATR'] = ohlc['ATR'].ewm(com=SmoothATR/(1.0-SmoothATR),adjust=False,ignore_na=False).mean()

Base volume 75.4222375821
               vol VALH
prc                    
4399.9  155.665410    b
4419.9  107.997114    l
4420.0  109.779323    h
4500.0  181.468892    l
4500.1  172.641538    h
4530.0  129.927579    b
4549.9   75.452013    l
4550.0  176.246091    h
4559.0   91.719592    b
4560.0  119.107749    b
4570.0   77.557590    b
4599.9   96.061596    b
4610.0  124.077951    b
4620.0  147.457668    b
4680.0   91.610746    b
4690.0   81.104130    b
4695.0   82.327040    b
4710.9  107.332670    l
4711.0  176.669275    h
4720.0  160.849542    b
4740.0  268.843665    b
4749.9  102.490849    l
4750.0  283.045225    h
4751.8  138.722626    l
4751.9   75.914540    h
4765.0   80.592112    b
4770.0  105.230938    b
4775.0   85.892499    b
4788.0  286.141148    b
4789.0   79.240978    b
4790.0   80.968416    b
4799.9  143.384611    l


Plotting result

In [46]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(x=hor_vol['vol'], y=hor_vol.index, orientation = 'h')

trace2 = go.Candlestick(open = ohlc.loc[:,'prc'].loc[:,'first'], 
                        high = ohlc.loc[:,'prc'].loc[:,'max'], 
                        low = ohlc.loc[:,'prc'].loc[:,'min'], 
                        close = ohlc.loc[:,'prc'].loc[:,'last'],
                        x = ohlc.index)
trace3 = go.Scatter(x = ohlc.index, y = ohlc['ATR']+hor_vol.index.min()-ohlc['ATR'].max())
trace4 = go.Scatter(x = ohlc.index, y = ohlc['smATR']+hor_vol.index.min()-ohlc['ATR'].max())

# formatting subplots
fig = tools.make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 2)
fig.append_trace(trace4, 1, 2)

fig['layout'].update(showlegend=False, xaxis=dict(rangeslider=dict(visible = False),showline=True,domain=[0, 0.3]), 
                     xaxis2=dict(showgrid=True,showline=True,domain=[0.3, 1]), yaxis=dict(showgrid=True), height=800)

# adding net levels on candlestick
fig['layout']['shapes'].append(  {
                            'xref':'x1', 'yref':'y1',
                            'type': 'line',
                            'x0': base_vol, 'y0': hor_vol.index.min(), 
                            'x1': base_vol, 'y1': hor_vol.index.max(),
                            'line': {
                                'color': 'rgb(55, 55, 55)',
                                'width': 1,
                                'dash': 'dot'
                            }
                        })

# adding net levels on candlestick
level_colors = {
    'h' : 'rgb(55, 255, 55)',
    'l' : 'rgb(255, 55, 55)',
    'b' : 'rgb(55, 55, 255)',
    'n' : 'rgb(55, 55, 55)',
}

for level in net_levels.index.values:    
    fig['layout']['shapes'].append(
                            {
                                'xref':'x2', 'yref':'y2',
                                'type': 'line',
                                'x0': ohlc.index.min(), 'y0': level, 
                                'x1': ohlc.index.max(), 'y1': level,
                                'line': {
                                    'color': level_colors[net_levels.loc[level,'VALH']],
                                    'width': 1,
                                    'dash': 'dot'
                                }
                            })

py.iplot(fig, filename='netter')

#py.iplot(dict(data=[trace3]), filename='netter_ATR')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]

